<a href="https://colab.research.google.com/github/saribasmetehan/BERT-BankingClassifier-Spacy/blob/main/Roadmap_for_just_a_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from datasets import load_dataset
import torch
import os

In [ ]:
dataset_name = "legacy-datasets/banking77"
dataset = load_dataset(dataset_name)

text_to_classify = "My cash in the ATM is still pending. I'm waiting"
classification_model_id = "saribasmetehan/BERT-BankingClassifier"
classifier = pipeline("text-classification", model=classification_model_id)

classification_result = classifier(text_to_classify)[0]
predicted_label = int(classification_result['label'].split("_")[1])

label_names = dataset['test'].features['label'].names
label_english = label_names[predicted_label]
output_file = f"{label_english}.txt"

filtered_data = [row["text"] for row in dataset["test"] if row["label"] == predicted_label]

with open(output_file, "w", encoding="utf-8") as f:
    f.write(f"Input text for classification:\n{text_to_classify}\n\n")
    f.writelines(text + "\n" for text in filtered_data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=0 if torch.cuda.is_available() else -1)

with open(output_file, "r", encoding="utf-8") as f:
    text_data = f.read()

summary = summarizer(text_data, max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)[0]['summary_text']

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
llm_model_name = "microsoft/Phi-3-mini-4k-instruct"
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

messages = [
    {"role": "system", "content": "You are an analyst who examines customer complaints of a bank. Evaluate the summary of complaints of many customers and provide insights and potential solutions."},
    {"role": "user", "content": f"Here is the summary of complaints related to the label '{label_english}':\n\n{summary}"},
]

output = pipe(messages, **generation_args)
generated_text = output[0]["generated_text"]

summary_output_dir = "output_directory"
os.makedirs(summary_output_dir, exist_ok=True)

results = [f"Label: {label_english}\n{generated_text}\n"]

output_file_path = os.path.join(summary_output_dir, "phi3_model_outputs.txt")
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write("\n\n".join(results))


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


In [ ]:
with open("/content/output_directory/phi3_model_outputs.txt", 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)

Label: pending_transfer
 Based on the summary of complaints related to the label 'pending_transfer', here are some insights and potential solutions:

1. Lack of communication: Customers are experiencing delays in their loan applications and transfers without any explanation. The bank should improve its communication with customers by providing timely updates and explanations for any delays or issues.

2. Technical issues: The pending transfer issue may be due to technical glitches or errors in the bank's system. The bank should investigate and resolve any technical issues that may be causing delays in the transfer process.

3. Verification process: Customers are double-checking their account details, but the transfer is still pending. The bank should review its verification process to ensure that it is efficient and accurate.

4. Payment due date: The transfer is marked as pending because the payment is due today. The bank should consider extending the payment due date for customers wh